In [21]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sleep_analysis_pipeline import *
from plotting_utils import *
from debugging_functions import *
%load_ext autoreload
%autoreload 2

input_dir = r'C:\Users\Experiment\Projects\video_conversions\pipeline_testing'

output_dir = setup_directories(input_dir)
print(f'Output directory: {output_dir}')

csv_file, video_file = find_files(input_dir)
print(f"Found CSV file: {csv_file}")
if video_file:
    print(f"Found video file: {video_file}")
else:
    print("No video file found")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Output directory: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output
Found CSV file: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000.csv
Found video file: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_labeled.mp4


In [2]:

# 3. Load DLC data
df_dlc = load_dlc_data(csv_file)
print(f"DLC data loaded: {df_dlc.shape[0]} rows, {len(df_dlc.columns)} columns")


DLC data loaded: 144500 rows, 81 columns


In [3]:

# 4. Get default bodyparts list
default_bodyparts = get_default_bodyparts()
print(f"Default bodyparts: {default_bodyparts}")

Default bodyparts: ['neck', 'mid_back', 'mouse_center', 'mid_backend', 'left_midside', 'right_midside', 'right_hip', 'left_ear', 'right_ear', 'left_ear_tip', 'right_ear_tip', 'left_shoulder', 'right_shoulder', 'head_midpoint']


In [4]:

# 5. Use select_available_bodyparts to find which bodyparts are available
final_bodyparts_list = select_available_bodyparts(df_dlc)
print(f"Final bodyparts list for analysis: {final_bodyparts_list}")


Final bodyparts list for analysis: ['neck', 'mid_back', 'mouse_center', 'mid_backend', 'left_midside', 'right_midside', 'right_hip', 'left_ear', 'right_ear', 'left_ear_tip', 'right_ear_tip', 'left_shoulder', 'right_shoulder', 'head_midpoint']


In [5]:

# 6. Process coordinates
df_dlc, filtered_x_coords, filtered_y_coords = process_bodypart_coordinates(df_dlc, final_bodyparts_list, likelihood_threshold)
print("Coordinate processing complete")


Applying likelihood filter and interpolation to selected bodyparts...
Displacement calculations complete.
Coordinate processing complete


In [6]:

# 7. Calculate speed
file_name = os.path.basename(csv_file)
df_dlc = calculate_speed(df_dlc, frame_rate, output_dir, file_name, final_bodyparts_list, save_plots=True)
print("Speed calculation complete")



Speed calculation complete.
DataFrame with speed data saved to: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_speed_analysis.csv

Generating speed plot... Individual parts: False, Smoothing: 0.25s
  Applying rolling average with window size: 15 frames (0.25s)
Speed plot saved to: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_speed_analysis_speed_plot_smoothed0.25s.png
Speed plots saved to: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output
Speed calculation complete


In [7]:

# 8. Display summary statistics for verification
print("\nSpeed Summary Statistics:")
print(df_dlc[('analysis', 'speed_pixels_per_second')].describe())



Speed Summary Statistics:
count    144499.000000
mean         27.264688
std          41.363934
min           0.027890
25%           5.840574
50%          12.430614
75%          32.637818
max        1629.878479
Name: (analysis, speed_pixels_per_second), dtype: float64


In [8]:
df_dlc = calculate_body_posture_metric(
    df_dlc, filtered_x_coords, filtered_y_coords, 
    final_bodyparts_list, output_dir, file_name, 
    save_plots=True
)
print("Body posture analysis complete")

Calculating body posture metric (average distance to median)...
Body posture metric calculated.
Body posture metric plot saved to: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_posture_analysis_body_posture_metric.png
Body posture metric plots saved to: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output
Body posture analysis complete


In [9]:
# 9. Calculate body movement derivative
df_dlc = calculate_body_movement_derivative(df_dlc, frame_rate, output_dir, file_name, save_plots=True)
print("Body movement derivative analysis complete")

Saved absolute body posture metric derivative plot: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_posture_analysis_body_posture_metric_abs_derivative.png
Body movement derivative calculation complete.
Body movement derivative analysis complete


In [10]:
# 10. Prepare coordinates for body axis calculation
bodypart_coordinate_sets = prepare_body_axis_coordinates(df_dlc, filtered_x_coords, filtered_y_coords, final_bodyparts_list)
print("Body axis coordinate preparation complete")


Body axis coordinate preparation complete.
Body axis coordinate preparation complete


In [11]:
df_dlc, df_midpoints_pca_raw = calculate_body_axis_pca(df_dlc, bodypart_coordinate_sets)

Body axis PCA calculation complete: successful in 144500/144500 frames (100.0%)
NaN counts in PCA results:
pca_dx                    0
pca_dy                    0
pca_mean_x                0
pca_mean_y                0
pca_explained_variance    0
dtype: int64


In [12]:
# 11. Calculate the body axis angles and create polar plot
df_dlc = calculate_and_plot_body_axis_angles(
    df_dlc, df_midpoints_pca_raw, filtered_x_coords, filtered_y_coords,
    frame_rate, output_dir, file_name, save_plots=True
)
print("Body axis angle analysis complete")

Calculated angles for PCA (midpoints_pca).
Saved polar angle plot: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_orientation_analysis_body_axis_angle_polar.png

--- Orientation Summary ---
Anatomical orientation was attempted using 'neck' (front) and 'mid_backend' (back).
Percentage of frames with anatomical orientation: 100.00%
Body axis angle calculation and plotting complete.
Body axis angle analysis complete


In [14]:
# 12. Calculate and plot smoothed body axis angles
df_dlc = plot_smoothed_body_axis_angles(
    df_dlc, df_midpoints_pca_raw, frame_rate, output_dir, file_name, save_plots=True
)
print("Smoothed body axis angle analysis complete")


Applying rolling window smoothing to body axis angles...
Applied rolling window smoothing with window size: 15 frames to angular data.
NaN count in raw angles: 0
NaN count in smoothed angles: 0
Saved smoothed polar angle plot: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_orientation_analysis_body_axis_angle_smoothed_polar.png
Smoothed body axis angle plotting complete.
Smoothed body axis angle analysis complete


In [17]:
# 13. Calculate angular velocity from smoothed body axis angles
df_dlc = calculate_angular_velocity(
    df_dlc, df_midpoints_pca_raw, frame_rate, output_dir, file_name, save_plots=True
)
print("Angular velocity analysis complete")



Calculating and plotting derivative of smoothed body axis angle...
Calculated angular velocity from smoothed angle (window=15 frames).
Saved angular velocity plot: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_orientation_analysis_angular_velocity.png
Angular velocity calculation and plotting complete.
Angular velocity analysis complete


In [19]:
# 14. Identify sleep bouts across all metrics
df_dlc, sleep_bouts_dict = identify_sleep_bouts_and_plot(
    df_dlc, df_midpoints_pca_raw, frame_rate, output_dir, file_name, save_plots=True
)
print("Sleep analysis complete")


Identifying and visualizing sleep bouts using multiple metrics...

--- Identifying Sleep Bouts Based on Speed ---
Identified 17 sleep bout(s) based on speed:
   start_time_s  end_time_s  duration_s  avg_speed_in_bout
0     27.633333   42.816667   15.183333          25.101659
1    362.266667  376.633333   14.366667          33.007027
2    535.716667  547.733333   12.016667          27.219687
3    547.900000  563.166667   15.266667          25.339116
4    682.716667  727.250000   44.533333          26.330362
...and 12 more bouts

--- Identifying Sleep Bouts Based on Body Posture Change ---
Identified 32 sleep bout(s) based on posture change:
   start_time_s  end_time_s  duration_s  avg_posture_change_in_bout
0     27.683333   42.483333   14.800000                    7.516529
1     42.883333   65.316667   22.433333                   11.858632
2     65.333333   86.366667   21.033333                    8.885853
3    107.366667  130.883333   23.516667                   11.120385
4    144.16

In [ ]:
if video_file:
    video_path = generate_sleep_analysis_video(
        df_dlc=df_dlc,
        df_midpoints_pca_raw=df_midpoints_pca_raw,
        sleep_bouts_dict=sleep_bouts_dict,
        smoothed_speed=df_dlc[('analysis', 'speed_smoothed')],
        body_movement_derivative=df_dlc[('analysis', 'posture_metric_abs_derivative')],
        angular_velocity=df_dlc[('analysis', 'absolute_angular_velocity')],
        smoothed_angle=df_midpoints_pca_raw['angle_y_deg_midpoints_pca_smoothed'],
        frame_rate=frame_rate,
        video_file=video_file,
        output_dir=output_dir,
        file_name=file_name,
        bodypart_coordinate_sets=bodypart_coordinate_sets,
        thresholds={
            'speed': 60,
            'posture': 60,
            'angular': 50
        }
    )
    
    if video_path:
        print("Video generation complete.")
    else:
        print("Video generation failed.")
else:
    print("No video file found for analysis. Skipping video generation.")


Generating comprehensive sleep analysis video...
Available sleep bout data:
- Speed-based: Yes
- Posture-based: Yes
- Angular velocity-based: Yes
Starting optimized comprehensive sleep analysis video creation: C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_sleep_analysis.mp4
Original matplotlib backend: agg
Temporarily switched matplotlib backend to: Agg
PCA data provided for arrow visualization.
Applied 15-frame smoothing to arrow visualization.
Median coordinates provided for overlay.
Added 17 speed-based sleep periods.
Added 32 posture-based sleep periods.
Added 20 angular velocity-based sleep periods.
Pre-calculating sleep period overlaps...
Pre-calculated 18 sleep overlap periods.


t:   0%|          | 117/144500 [00:34<5:53:23,  6.81it/s, now=None]

Writing final video to C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_sleep_analysis.mp4...
Moviepy - Building video C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_sleep_analysis.mp4.
Moviepy - Writing video C:\Users\Experiment\Projects\video_conversions\pipeline_testing\sleep_pipeline_output\fullDLC_resnet50_downsampled_trialJul11shuffle1_150000_sleep_analysis.mp4



Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x000001F369F9D510>
Traceback (most recent call last):
  File "c:\Users\Experiment\Miniconda3\envs\speed_analysis\lib\site-packages\matplotlib\transforms.py", line 222, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id(self): pop(k))
KeyboardInterrupt: 
